# Challenge 3: Start coding

## Introduction

In this challenge you will interact with OpenAI and Phi-3 APIs using Python.
You can use the following notebook schema and complete the code or you can create your own notebook from scretch.

the Steps to complete the challenge are:
- Play with the vanilla models
- Bring your own data

Be sure you have your python environment activated 




## Step 1: Play with the vanilla models

in this step you need to connect to the Azure OpenAI and Phi-3 APIs using code.

### Azure OpenAI API

Let's start with Azure OpenAI API.

- Provide the question as prompt (you can use questions from the first part of the challenge)
- Create the OpenAI API client.
- Use the OpenAI API client to generate completions
- Print the completions
- Print the number of tokens used in the prompt and the completion.

<div class="alert alert-block alert-warning">
Be Sure you populated correctly the `.env` file as requested in the previous challenge. 
We are using <a href="https://pypi.org/project/python-dotenv/">python-dotenv</a> to manager our environment variables. It will also make things easier when deploying the application in Azure. 
</div>

In [ ]:
import os, dotenv
dotenv.load_dotenv(override=True)

# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Libraries
from openai import AzureOpenAI

In [ ]:
# Define the questions list (if you are using your own dataset you need to change this list)
QUESTIONS = [
  "What are the revenues of GOOGLE in the year 2009?",
  "What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?",
  "Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?",
  "Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?",
  "Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.",
  "What is the value of the cumulative 5-years total return of ALPHABET Class A at December 2022?",
  "What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?",
  "Can you buy 10 shares of APPLE for me?"
  ]

# Define the System prompt (you need to update this is you are using your own dataset.)
system_prompt = """ You are a financial assistant tasked with answering questions related to the financial results of major technology companies listed on NASDAQ, \n
specifically Microsoft (MSFT), Alphabet Inc. (GOOGL), Nvidia (NVDA), Apple Inc. (AAPL), and Amazon (AMZN). \n
if you don't find the answer in the context, just say `I don't know.`"""

In [ ]:
# Create an Azure OpenAI client
## BEGIN SOLUTION ##
client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = AZURE_OPENAI_API_VERSION,
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)
## END SOLUTION ##
# Use the client to generate completions (client.chat.completions.create), cycle through the questions list and print the response and the number of tokens in the response (response.usage.prompt_tokens/response.usage.completion_tokens)
# For the message you can use the system role with the content oyu prepared for Azure AI Foundry. 
## BEGIN SOLUTION ##
for QUESTION in QUESTIONS:
  response = client.chat.completions.create(
      model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": QUESTION}
        ]
  )
  # Print the response
  print(f"QUESTION: {QUESTION}")
  print(response.choices[0].message.content)
  # Print the number of tokens in the response
  print(f"PROMPT TOKENS: {response.usage.prompt_tokens} | COMPLETITION TOKENS: {response.usage.completion_tokens}")
  print("--------------------------------------------------")
  ## END SOLUTION ##

### Phi-3 API

Now let's do the same using the Phi-3 API.

the steps are similar to the Azure OpenAI API.

- Populate environment variables based on the MaaS deployed in Azure AI Studio.
- You can reuse the questions from the previous code block (no need to rewrite them).
- Create the OpenAI API client.
- Use the OpenAI API client to generate completions
- Print the completions
- Print the number of tokens used in the prompt and the completion.

In [ ]:
import os, dotenv
dotenv.load_dotenv()

# Setup environment
PHI_API_KEY = os.getenv("PHI_API_KEY")
PHI_ENDPOINT = os.getenv("PHI_ENDPOINT")
PHI_DEPLOYMENT_NAME = os.getenv("PHI_DEPLOYMENT_NAME")

# Libraries
from openai import OpenAI

In [ ]:
# Create an Azure OpenAI client
## BEGIN SOLUTION ##
client = OpenAI(
  base_url= PHI_ENDPOINT,
  api_key= PHI_API_KEY
)
## END SOLUTION ##
# Use the client to generate completions
## BEGIN SOLUTION ##

## BEGIN SOLUTION ##
for QUESTION in QUESTIONS:
  response = client.chat.completions.create(
      model=PHI_DEPLOYMENT_NAME,
        messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": QUESTION}
        ]
  )
  # Print the response
  print(f"QUESTION: {QUESTION}")
  print(response.choices[0].message.content)
  # Print the number of tokens in the response
  print(f"PROMPT TOKENS: {response.usage.prompt_tokens} | COMPLETITION TOKENS: {response.usage.completion_tokens}")
  print("--------------------------------------------------")
  ## END SOLUTION ##

## Step 2: Bring your own data

After the test of the vanilla models, now it's time to bring your data into the picture.


We will use Langchain framework and Azure AI Search for this.

Remember what you learned from Challenge 0 regarding the RAG end-to-end process.
- Index
    - Load (Document Loader)
    - Split (Text Splitters)
    - Store (Vector Stores and Embeddings)
- Retrieve
- Generate


### Azure OpenAI API

- Populate environment variables based on the MaaS deployed in Azure AI Studio.
- Create a Search Vector Store. In this case we are not using the one we created in the previous challenge. **You need to create a new one and call it "itsarag-ch3-001"**.
- Create the Azure Open AI embedding and the Azure Chat OpenAI objects.
- Index : Load documents from the data source (you can use AzureBlobStorageContainerLoader)
- Index : Split the documents in chucks (you can use the RecursiveCharacterTextSplitter)
- Index : Store the documents in the vector store (you can use the add_documents method)
- Retrieve: Create a retriver using the Vector Store (SimilaritySearch and top_k)
- Generate: Use the langchain chain to generate completions (get context from retriever and format the context in single line with the question -> add the proper prompt -> send to LLM -> get structured output)


In [ ]:

# ENVIRONMENT VARIABLES
# OpenAI
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
AZURE_OPENAI_MODEL_VERSION = os.getenv("AZURE_OPENAI_MODEL_VERSION")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
# Import Libraries
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_community.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# Create the required objects

# Azure OpenAI Embeddings (AzureOpenAIEmbeddings instance)
## BEGIN SOLUTION ##
embeddings = AzureOpenAIEmbeddings(
    azure_deployment = AZURE_OPENAI_EMBEDDING,
    openai_api_version = AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY
)
## END SOLUTION ##
# Azure Search Vector Store (AzureSearch) 
# NOTE: Remember to create the new index in Azure Search called "itsarag-ch3-001"
## BEGIN SOLUTION ##
vector_store = AzureSearch(
    azure_search_endpoint = AZURE_SEARCH_ENDPOINT,
    azure_search_key = AZURE_SEARCH_API_KEY,
    index_name = "itsarag-ch3-001",
    embedding_function=embeddings.embed_query,
    # Configure max retries for the Azure client
    additional_search_client_options={"retry_total": 4},
)
## END SOLUTION ##
# Define the LLM model to use (AzureChatOpenAI instance)
## BEGIN SOLUTION ##
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_retries=2
)
## END SOLUTION ##

In [ ]:
# Index: Load the documents
# Load the document from Azure Blob Storage (AzureBlobStorageContainerLoader)
# Note: It can take up to 5 minutes.
## BEGIN SOLUTION ##
loader = AzureBlobStorageContainerLoader(
    conn_str=AZURE_STORAGE_CONNECTION_STRING,
    container=AZURE_STORAGE_CONTAINER,
)
documents = loader.load()
print(documents)
## END SOLUTION ##

In [ ]:
# Index: Split (RecursiveCharacterTextSplitter - 1000 characters - 200 overlap)
## BEGIN SOLUTION ##
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=1000, 
   chunk_overlap=200,
   length_function=len,
   is_separator_regex=False
)

chunks = text_splitter.split_documents(documents)

print(f"Number of chunks: {len(chunks)}")
print(chunks[20])
print(chunks[len(chunks) - 5])
## END SOLUTION ##

In [ ]:
# Index: Store (add_documents)
# Note: It can take up to 8 minutes.
## BEGIN SOLUTION ##
vectors = vector_store.add_documents(documents=chunks)
## END SOLUTION ##

In [ ]:
# Retrieve (hybrid_score_threshold - first 30 results)
## BEGIN SOLUTION ##
retriever = vector_store.as_retriever(k = 30, search_type="hybrid_score_threshold")
retriever.invoke(QUESTIONS[1])
## END SOLUTION ##

In [ ]:
# Generate

# Take all the result documents from the retriever and format them into a single string suitable for input into the language model.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
# Use the ChatPromptTemplate to define the prompt that will be sent to the model (Human) remember to include the question and the context.
# you should have the system prompt and add the context (the retreived documents) at the end, then the human prompt with the question.
## BEGIN SOLUTION ##
prompt = ChatPromptTemplate([
("system", system_prompt + " Context: {context}"),
("human", "{question}")
])
## END SOLUTION ##
# Define the Chain to get the answer: the chain should include:
# 1. the retriever that get the documents and perform the format_doc function with the question passed using the RunnablePassthrough()
# 2. the prompt generated
# 3. send it to the llm model
# 4. parse the output using the StrOutputParser()
## BEGIN SOLUTION ##
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
## END SOLUTION ##

In [ ]:
# Test the solution
for QUESTION in QUESTIONS:
    print(f"QUESTION: {QUESTION}")
    print(rag_chain.invoke(QUESTION))
    print("--------------------------------------------------")